In [2]:
from os import path,listdir
from datetime import datetime,timedelta
import time
import re
import pandas as pd
from IPython.display import display
import numpy as np
import random
import pytz
#from tqdm import tqdm
import bz2
import json
import glob
local_tz = 'UTC'
from api.data_collector import DataCollector
from api.bind import *

In [1]:
a='votes_8805230_2021-01-30-2221'
a.split('_')[1][:7]

'8805230'

In [4]:
dp=DataCollector()
#df_sofa=dp._provide_sofa()
df_sofa=dp._load_data()
df_op=dp._provide_op()
#df_elo=dp._provide_elo()
df_op_=df_op[df_op['ds']>=df_sofa.ds.min()]

Empty DataFrame
Columns: [team]
Index: []


In [7]:
ts=pytz.timezone(local_tz).localize(datetime.today()-timedelta(days=20))
df_op_=df_op[df_op['ds']>=ts]
df_sofa=df_sofa[df_sofa['ds']>=ts]
df_sofa_binded, df_sofa_ = bind_full(df_sofa,df_op_)
df_sofa_binded=bind_iteration('FIRST',df_sofa_binded,df_sofa_, df_op_)
df_sofa_binded=bind_iteration('SECOND',df_sofa_binded,df_sofa_, df_op_)
df_sofa_binded=bind_iteration('THIRD',df_sofa_binded,df_sofa_, df_op_)

IN: Sofa=(806, 46), OP=(43392, 22)
BOTH teams step: Binded=(295, 53), Total=(295, 53), Rest=(789, 48)
First team step: Binded=(194, 53), Total=(489, 53), Rest=(773, 48), Excluded=(7, 48)
Second team step: Binded=(109, 53), Total=(598, 53), Rest=(773, 48), Excluded=(9, 48)
**** FIRST ITERATION ****
T1 merged:  (773, 49)
T2 merged:  (773, 50)
IN: 773, BOTH: 7, ONLY T1: 37, ONLY T2: 30, NO BINDS: 699, OUT: 773
IN: Sofa=(7, 50), OP=(43392, 22)
Both teams step, exact dates: Binded=(0, 54), Total=(0, 54), Rest=(7, 51)
Both teams step, within a day: Binded=(0, 54), Total=(0, 54), Rest=(7, 51)
(33, 53)
IN: Sofa=(37, 50), OP=(43392, 22)
First team step, exact dates: Binded=(53, 54), Total=(53, 54), Rest=(34, 51)
First team step, within a day: Binded=(0, 54), Total=(3, 54), Rest=(34, 51)
(36, 53)
IN: Sofa=(30, 50), OP=(43392, 22)
Second team step, exact dates: Binded=(156, 54), Total=(156, 54), Rest=(25, 51)
Second team step, within a day: Binded=(0, 54), Total=(5, 54), Rest=(25, 51)
(41, 53)
sa

In [ ]:
df_teams=pd.read_csv('data/teams.csv', index_col=None).sort_values(by='op_t')
mask = df_teams.duplicated(subset=['country','op_t'], keep=False)
display(df_teams[mask])

In [ ]:
df_elo.team=df_elo.team.str.lower()
df_elo=df_elo.rename(columns={'country':'code'})
df_elo_teams=df_elo[['team','code','id']].drop_duplicates().sort_values(by='team')
df_countries=pd.read_csv('data/elo/countries.csv', index_col=None)
df_elo_teams=df_elo_teams.merge(df_countries, on='code', how='left')
df_elo_teams.to_csv('data/elo/elo_teams.csv', index=False)
df_elo_teams['first']=df_elo_teams['team'].apply(lambda x: x.split(' ')[0])
df_elo_teams['last']=df_elo_teams['team'].apply(lambda x: x.split(' ')[-1])

In [ ]:
df_teams=pd.read_csv('data/teams.csv', index_col=None)
df_teams

In [ ]:
df_elo_merged=df_elo.merge(df_teams[['id','tid']], on='id', how='left').drop_duplicates()
df_elo_merged=df_elo_merged.dropna()
df_elo_merged

# Today

In [ ]:
dp=DataCollector()


# Load data
## SofaScore

In [2]:
df_countries=pd.read_csv('data/sofa/countries.csv', index_col=None)
df_countries['Name']=df_countries['Name'].str.lower()
df_countries.columns=['country','countryCode']

chars0=['ó','é','í','ş','ã','İ','ğ','ç','ü','É','â','Ç','õ','ł','ą','Ś','ø','ń','ț','å','Å','ß', 'æ', 'Ž','ş', 'ə','Ö','ı','á','î','ñ','ö','ź','ú','è','Ł','ę','Ş','ä','ë','ô','ș','ū','č','Š','Þ','ė','Ä','ă','ì','š','i','ć','ň','ž','ư','ơ','ê','à','ð','ő','Ü','ý','ď','Á','ř','Č','Ú']
chars1=['o','e','i','s','a','I','g','c','u','E','a','C','o','l','a','s','o','n','t','a','A','ss','ae','Z','sh','a','O','i','a','i','n','o','z','u','e','L','e','S','a','e','o','s','u','c','S','P','e','A','a','i','s','i','c','n','z','u','o','e','a','d','o','U','y','d','A','r','C','U']
dicUnicode2En=dict(zip(chars0, chars1))

df_ss=pd.read_csv('data/sofa/matches_done.csv', index_col=None)
df_ss['ts']=pd.to_datetime(df_ss['ts'])
df_ss=df_ss.merge(df_countries, on='country', how='left')
df_ss.loc[df_ss['country']=='england','countryCode']='GB'
df_ss.loc[df_ss['country']=='scotland','countryCode']='GB'
df_ss.loc[df_ss['country']=='czech-republic','countryCode']='CZ'
df_ss.loc[df_ss['country']=='russia','countryCode']='RU'
df_ss.loc[df_ss['country']=='usa','countryCode']='US'
df_ss.loc[df_ss['homeTeamShort']=='???','homeTeamShort']='AEK Athens'
df_ss.loc[df_ss['awayTeamShort']=='???','awayTeamShort']='AEK Athens'
df_ss['homeTeamShortLow']=df_ss['homeTeamShort'].replace(dicUnicode2En, regex=True).replace('[^a-zA-Z0-9 ]', '', regex=True).str.lower()
df_ss['awayTeamShortLow']=df_ss['awayTeamShort'].replace(dicUnicode2En, regex=True).replace('[^a-zA-Z0-9 ]', '', regex=True).str.lower()
countries_of_interest=sorted(list(df_ss.country.unique()))
countries_of_interest+=['']

## Betfair

In [ ]:
df_bf=pd.read_csv('data/bf/bf_matches.csv', index_col=None)
df_bf=df_bf[~df_bf['halfTime'].isna()]
df_bf=df_bf[df_bf['halfTime']!='0']
df_bf['inplayTime']=pd.to_datetime(df_bf['inplayTime'])
df_bf['home_name_low']=df_bf['home_name'].replace('[^a-zA-Z0-9 ]', '', regex=True).str.lower()
df_bf['away_name_low']=df_bf['away_name'].replace('[^a-zA-Z0-9 ]', '', regex=True).str.lower()

df_countries=pd.read_csv('data/sofa/countries.csv', index_col=None)
df_countries['Name']=df_countries['Name'].str.lower()
df_countries.columns=['country','countryCode']
df_bf=df_bf.merge(df_countries, on='countryCode', how='left')

df_bf=df_bf[df_bf['countryCode']!='CS']
df_bf.loc[df_bf['countryCode'].isna(),'country']=''
countries_replacement={'united kingdom':'england', 'russian federation':'russia','united states':'usa','czech republic':'czech-republic','korea, republic of':'south-korea' }
df_bf['country']=df_bf['country'].replace(countries_replacement)
df_bf=df_bf.loc[df_bf['country'].isin(countries_of_interest)]

## Fbref

In [ ]:
def get_country(x):
    if x==0:
        return ''
    res=df_countries.loc[df_countries['comps'].str.contains(str(int(x))),'name']
    if len(res.index)>0:
        return res.values[0]
    else:
        return ''
        
df_countries=pd.read_csv('data/fbref/countries.csv', index_col=None)
df_countries['name']=df_countries['name'].str.lower()
#df_countries.columns=['country','countryCode']

df_fbref=pd.read_csv('data/fbref/matches_full.csv', index_col=None)
df_fbref=df_fbref[~df_fbref['ds_venue'].isna()]
df_fbref['ts']=df_fbref['ds_venue'].apply(lambda x: datetime.utcfromtimestamp(x))

df_fbref['home_name_low']=df_fbref['team1'].replace(dicUnicode2En, regex=True).replace('[^a-zA-Z0-9 ]', '', regex=True).str.lower()
df_fbref['away_name_low']=df_fbref['team2'].replace(dicUnicode2En, regex=True).replace('[^a-zA-Z0-9 ]', '', regex=True).str.lower()
df_fbref['country_id']=df_fbref['country_id'].fillna(0)
df_fbref['country']=df_fbref['country_id'].apply(lambda x: get_country(x))

In [ ]:
pd.DataFrame([df_fbref.isna().sum(),df_fbref.isna().sum()/1230 ]).T

## OP

In [68]:
df_op=pd.read_csv('data/op/matches1.csv', index_col=None)
df_op['home_name_low']=df_op['t1'].replace('[^a-zA-Z0-9 ]', '', regex=True).str.lower()
df_op['away_name_low']=df_op['t2'].replace('[^a-zA-Z0-9 ]', '', regex=True).str.lower()
df_op['ts']=pd.to_datetime(df_op['ds'], format='%y/%m/%d %H:%M')

t1=df_op[['country','t1']].rename(columns={'t1': 'name'})
t2=df_op[['country','t2']].rename(columns={'t2': 'name'})

df_teams=pd.DataFrame(pd.concat([t1,t2], axis=0)).drop_duplicates()

df_countries=pd.read_csv('data/op/countries.csv', index_col=None)
countries=dict(zip(df_countries.abbr, df_countries.name.str.lower()))

df_teams['cnt']=df_teams['name'].apply(lambda x: x.split('(')[1].replace(')','').strip() if '(' in x else '')
df_teams1=df_teams.loc[df_teams['cnt'].str.len()>1]
df_teams2=df_teams.loc[df_teams['cnt'].str.len()<1]
df_teams1['country']=df_teams1.cnt.apply(lambda x: countries[x] if x in countries else 'other')
df_teams=pd.concat([df_teams1,df_teams2],axis=0)
df_teams['clear']=df_teams.name.apply(lambda x: x.split('(')[0].strip())
df_teams

,country,name,cnt,clear
46,chile,Colo Colo (Chi),Chi,Colo Colo
80,austria,Rapid Vienna (Aut),Aut,Rapid Vienna
287,france,Bordeaux W (Fra),Fra,Bordeaux W
361,spain,Melilla (Esp),Esp,Melilla
401,spain,Numancia (Esp),Esp,Numancia
...,...,...,...,...
143800,spain,RSD Alcala,,RSD Alcala
144125,world,Hienghene,,Hienghene
144223,spain,Chinato,,Chinato
145021,world,Bucaspor,,Bucaspor


In [21]:
df_clear=pd.DataFrame(df_teams.clear.unique(), columns=['clear'])
df_clear['tid'] = df_clear.index
df_clear['tid'] = df_clear.tid.apply(lambda x: '{:04.0f}'.format(x))
df_teams=df_teams.merge(df_clear, on=['clear'], how='left')
df_teams

,clear,tid
0,Boreham Wood,0000
1,Dortmund,0001
2,Barcelona SC,0002
3,Cambridge Utd,0003
4,Esteghlal F.C.,0004
...,...,...
4546,Coleraine,4546
4547,Conquense,4547
4548,Loures,4548
4549,Qandi,4549


In [25]:
df_teams=df_teams.merge(df_clear, on=['clear'], how='left')

In [29]:
gr=df_teams.groupby(['tid']).count()
gr.loc[gr['clear']>2]

,name,clear
tid,,
0165,3,3
0619,3,3
0798,3,3
0846,3,3
0909,3,3


In [31]:
df_teams[df_teams.tid=='0619']

,name,clear,tid
622,River Plate,River Plate,0619
2453,River Plate (Arg),River Plate,0619
3995,River Plate (Uru),River Plate,0619


# Binding
## SS - BF

In [20]:
cc_to_empty=['africa','asia','europe','north-central-america','south-america','world']
def slice_df(df, cc, str1, str2, type='00'):
    if cc in cc_to_empty:
        cc=''
    if type=='00':
        return df.loc[(df['country']==cc) & (df['home_name_low']==str1) & (df['away_name_low']==str2)]
    elif type=='10':
        return df.loc[(df['country']==cc) & (df['home_name_low'].str.contains(str1)) & (df['away_name_low']==str2)]
    elif type=='01':
        return df.loc[(df['country']==cc) & (df['home_name_low']==str1) & (df['away_name_low'].str.contains(str2))]
    else:
        return df.loc[(df['country']==cc) & (df['home_name_low'].str.contains(str1)) & (df['away_name_low'].str.contains(str2))]

In [ ]:
binds=[]
d= datetime(2015, 5, 1)
end_date= datetime(2020, 11, 1)
while d<end_date:
    print(d)
    dloc=pytz.timezone(local_tz).localize(d)
    df_ss_day=df_ss[(df_ss['ts']>=dloc) & (df_ss['ts']<dloc+timedelta(days=1))]
    df_bf_day=df_bf[(df_bf['inplayTime']>=d) & (df_bf['inplayTime']<d+timedelta(days=1))]
    for row in df_ss_day.itertuples(index=False):
        home_parts=row.homeTeamShortLow.split(' ')
        away_parts=row.awayTeamShortLow.split(' ')
        df_bf_slice=slice_df(df_bf_day, row.country, row.homeTeamShortLow, row.awayTeamShortLow, type='00')
        if len(df_bf_slice.index)!=1:
            df_bf_slice=slice_df(df_bf_day, row.country, row.homeTeamShortLow, row.awayTeamShortLow, type='10')
        if len(df_bf_slice.index)!=1:
            df_bf_slice=slice_df(df_bf_day, row.country, row.homeTeamShortLow, row.awayTeamShortLow, type='01')
        if len(df_bf_slice.index)!=1:
            df_bf_slice=slice_df(df_bf_day, row.country, row.homeTeamShortLow, row.awayTeamShortLow, type='11')
        if len(df_bf_slice.index)!=1 and ' ' in row.homeTeamShortLow:
            homeFirst=home_parts[0]
            homeLast=home_parts[-1]
            if len(homeFirst)>2:
                df_bf_slice=slice_df(df_bf_day, row.country, homeFirst, row.awayTeamShortLow, type='10')
            if len(df_bf_slice.index)!=1 and len(homeLast)>2:
                df_bf_slice=slice_df(df_bf_day, row.country, homeLast, row.awayTeamShortLow, type='10')
        if len(df_bf_slice.index)!=1 and ' ' in row.awayTeamShortLow:
            awayFirst=away_parts[0]
            awayLast=away_parts[-1]
            if len(df_bf_slice.index)!=1 and len(awayFirst)>2:
                df_bf_slice=slice_df(df_bf_day, row.country, row.homeTeamShortLow, awayFirst, type='01')
            if len(df_bf_slice.index)!=1 and len(awayLast)>2:
                df_bf_slice=slice_df(df_bf_day, row.country, row.homeTeamShortLow, awayLast, type='01')
            if len(df_bf_slice.index)!=1 and ' ' in row.homeTeamShortLow:
                homeFirst=home_parts[0]
                homeLast=home_parts[-1]
                if len(df_bf_slice.index)!=1 and len(homeFirst)>2 and len(awayFirst)>2:
                    df_bf_slice=slice_df(df_bf_day, row.country, homeFirst, awayFirst, type='11')
                if len(df_bf_slice.index)!=1 and len(homeFirst)>2 and len(awayLast)>2:
                    df_bf_slice=slice_df(df_bf_day, row.country, homeFirst, awayLast, type='11')
                if len(df_bf_slice.index)!=1 and len(awayLast)>2 and len(awayFirst)>2:
                    df_bf_slice=slice_df(df_bf_day, row.country, homeLast, awayFirst, type='11')
                if len(df_bf_slice.index)!=1 and len(homeLast)>2 and len(awayLast)>2:
                    df_bf_slice=slice_df(df_bf_day, row.country, homeLast, awayLast, type='11')
        if len(df_bf_slice.index)==1:
            eventId,home_id,away_id,bf_ht,bf_at=df_bf_slice.iloc[0][['eventId','home_id','away_id','home_name','away_name']]
            print(f'found: {row.homeTeamShort}/{bf_ht} - {row.awayTeamShort}/{bf_at}')
            binds.append({
                'bf_home_name':bf_ht,
                'bf_away_name':bf_at,
                'bf_home_id':home_id,
                'bf_away_id':away_id,
                'bf_eventId':eventId,
                'ss_home':row.homeTeamShort,
                'ss_away':row.awayTeamShort,
                'ss_id':row.id
            })
    #
    d+=timedelta(days=1)
    #break
pd.DataFrame(binds).to_csv('data/binds.csv', index=False)

## SS - Fbref

In [82]:
def bind_full(df_source,df_target, ds,de,target,cols, isWide=False):
    binds=[]
    d= ds
    while d<de:
        #print(d)
        dloc=pytz.timezone(local_tz).localize(d)
        ds_src=dloc-timedelta(days=1) if isWide else dloc
        de_src=dloc+timedelta(days=2) if isWide else dloc+timedelta(days=2)
        ds_tgt=d-timedelta(days=1) if isWide else d
        de_tgt=d+timedelta(days=2) if isWide else d+timedelta(days=2)

        df_src=df_source[(df_source['ts']>=ds_src) & (df_source['ts']<de_src)]
        df_tgt=df_target[(df_target['ts']>=ds_tgt) & (df_target['ts']<de_tgt)]
        for row in df_src.itertuples(index=False):
            home_parts=row.homeTeamShortLow.split(' ')
            away_parts=row.awayTeamShortLow.split(' ')
            df_tgt_slice=slice_df(df_tgt, row.country, row.homeTeamShortLow, row.awayTeamShortLow, type='00')
            if len(df_tgt_slice.index)!=1:
                df_tgt_slice=slice_df(df_tgt, row.country, row.homeTeamShortLow, row.awayTeamShortLow, type='10')
            if len(df_tgt_slice.index)!=1:
                df_tgt_slice=slice_df(df_tgt, row.country, row.homeTeamShortLow, row.awayTeamShortLow, type='01')
            if len(df_tgt_slice.index)!=1:
                df_tgt_slice=slice_df(df_tgt, row.country, row.homeTeamShortLow, row.awayTeamShortLow, type='11')
            if len(df_tgt_slice.index)!=1 and ' ' in row.homeTeamShortLow:
                homeFirst=home_parts[0]
                homeLast=home_parts[-1]
                if len(homeFirst)>2:
                    df_tgt_slice=slice_df(df_tgt, row.country, homeFirst, row.awayTeamShortLow, type='10')
                if len(df_tgt_slice.index)!=1 and len(homeLast)>2:
                    df_tgt_slice=slice_df(df_tgt, row.country, homeLast, row.awayTeamShortLow, type='10')
            if len(df_tgt_slice.index)!=1 and ' ' in row.awayTeamShortLow:
                awayFirst=away_parts[0]
                awayLast=away_parts[-1]
                if len(df_tgt_slice.index)!=1 and len(awayFirst)>2:
                    df_tgt_slice=slice_df(df_tgt, row.country, row.homeTeamShortLow, awayFirst, type='01')
                if len(df_tgt_slice.index)!=1 and len(awayLast)>2:
                    df_tgt_slice=slice_df(df_tgt, row.country, row.homeTeamShortLow, awayLast, type='01')
                if len(df_tgt_slice.index)!=1 and ' ' in row.homeTeamShortLow:
                    homeFirst=home_parts[0]
                    homeLast=home_parts[-1]
                    if len(df_tgt_slice.index)!=1 and len(homeFirst)>2 and len(awayFirst)>2:
                        df_tgt_slice=slice_df(df_tgt, row.country, homeFirst, awayFirst, type='11')
                    if len(df_tgt_slice.index)!=1 and len(homeFirst)>2 and len(awayLast)>2:
                        df_tgt_slice=slice_df(df_tgt, row.country, homeFirst, awayLast, type='11')
                    if len(df_tgt_slice.index)!=1 and len(awayLast)>2 and len(awayFirst)>2:
                        df_tgt_slice=slice_df(df_tgt, row.country, homeLast, awayFirst, type='11')
                    if len(df_tgt_slice.index)!=1 and len(homeLast)>2 and len(awayLast)>2:
                        df_tgt_slice=slice_df(df_tgt, row.country, homeLast, awayLast, type='11')
            if len(df_tgt_slice.index)==1:
                if len(cols)==3:
                    mid,home_team,away_team=df_tgt_slice.iloc[0][cols]
                else:
                    mid,home_id,away_id,home_team,away_team=df_tgt_slice.iloc[0][cols]

                #print(f'found: {row.homeTeamShort}/{home_team} - {row.awayTeamShort}/{away_team}')
                binds.append({
                    'target_home_name':home_team,
                    'target_away_name':away_team,
                    'target_home_id':home_id if len(cols)>3 else home_team,
                    'target_away_id':away_id if len(cols)>3 else away_team,
                    'target_mid':mid,
                    'ss_home':row.homeTeamShort,
                    'ss_away':row.awayTeamShort,
                    'ss_id':row.id
                })
        #
        d+=timedelta(days=1)
    df_binds=pd.DataFrame(binds)
    df_binds.to_csv(f'data/binds_ss_{target}.csv', index=False)
    return df_binds

In [30]:
df_binds=bind_full(df_ss,df_fbref, datetime(2015, 1, 1), datetime(2020, 12, 1),'fbref',['mid','tid1','tid2','team1','team2'])

NameError: name 'df_fbref' is not defined

In [83]:
def full_semi_none(df_binds, df_ss, ds):
    df_binds.loc[df_binds['target_home_id'].isna(),'target_home_id']=''
    df_binds.loc[df_binds['target_away_id'].isna(),'target_away_id']=''
    ds = pytz.timezone(local_tz).localize(ds)
    ss=df_ss.loc[df_ss['ts']>=ds]

    #df_binds=pd.read_csv('data/binds.csv', index_col=None)

    df_ss_binded=ss.merge(df_binds, left_on=['id','homeTeamShort', 'awayTeamShort'] , right_on=['ss_id','ss_home','ss_away'], how='inner')
    df_ss_binded=df_ss_binded.drop_duplicates()
    df_semi = ss.loc[~(ss['id'].isin(df_ss_binded['id']))]
    df_semi=df_semi.merge(df_binds[['target_home_name', 'target_home_id', 'ss_home']].drop_duplicates(), left_on=['homeTeamShort'] , right_on=['ss_home'], how='left')
    df_semi=df_semi.merge(df_binds[['target_away_name', 'target_away_id', 'ss_away']].drop_duplicates(), left_on=['awayTeamShort'] , right_on=['ss_away'], how='left')
    df_not=df_semi.loc[(df_semi['target_home_id'].isna()) & (df_semi['target_away_id'].isna())]
    df_semi = df_semi.loc[~(df_semi['id'].isin(df_not['id']))]
    df_semi=df_semi.drop_duplicates()
    df_semi.loc[df_semi['target_home_id'].isna(),'target_home_id']=''
    df_semi.loc[df_semi['target_away_id'].isna(),'target_away_id']=''
    print(' full: {}, semi: {}, non: {}'.format(len(df_ss_binded.index),len(df_semi.index),len(df_not.index)) )
    return df_ss_binded,df_semi,df_not

def bind_semi(df_semi,df_target,cols, isWide=False):
    binds=[]
    binded_total=0
    for row in df_semi.itertuples(index=False):
        d=row.ts.replace(tzinfo=None)
        ds_tgt=d-timedelta(days=1) if isWide else d
        de_tgt=d+timedelta(days=2) if isWide else d+timedelta(days=2)
        if row.target_home_id!='':
            #print(0,row.target_home_id)
            
            df_target_slice=df_target[(df_target['ts']>=ds_tgt) & (df_target['ts']<de_tgt) & (df_target[cols[1]]==row.target_home_id)]
            if len(df_target_slice.index)==1:
                ssid, sshid, sshname, ssaname = (row.id,row.target_home_id,row.homeTeamShort,row.awayTeamShort)
                target_id, target_hid, target_aid, target_hname, target_aname = (df_target_slice.iloc[0][cols]).values
                #print('Found on home id={}, teams: {}/{} vs {}/{}, match ids {}-{}'.format(sshid, sshname,target_hname,ssaname,target_aname,ssid,target_id ))
                binds.append({
                    'target_home_name':target_hname,
                    'target_away_name':target_aname,
                    'target_home_id':target_hid,
                    'target_away_id':target_aid,
                    'target_mid':target_id,
                    'ss_home':sshname,
                    'ss_away':ssaname,
                    'ss_id':ssid
                })
                binded_total+=1
        else:
            #print(1,row.target_away_id)
            df_target_slice=df_target[(df_target['ts']>=ds_tgt) & (df_target['ts']<de_tgt) & (df_target[cols[2]]==row.target_away_id)]
            if len(df_target_slice.index)==1:
                ssid, ssaid, sshname, ssaname = (row.id,row.target_away_id,row.homeTeamShort,row.awayTeamShort)
                target_id, target_hid, target_aid, target_hname, target_aname = (df_target_slice.iloc[0][cols]).values
                #print('Found on away id={}, teams: {}/{} vs {}/{}, match ids {}-{}'.format(ssaid, sshname,target_hname,ssaname,target_aname,ssid,target_id ))
                binds.append({
                    'target_home_name':target_hname,
                    'target_away_name':target_aname,
                    'target_home_id':target_hid,
                    'target_away_id':target_aid,
                    'target_mid':target_id,
                    'ss_home':sshname,
                    'ss_away':ssaname,
                    'ss_id':ssid
                })
                binded_total+=1
    print (f'Done! {binded_total} matches binded')
    return pd.DataFrame(binds)

In [ ]:
#df_binds=pd.read_csv('data/binds_ss_fbref.csv', index_col=None)
df_ss_binded,df_semi,df_not=full_semi_none(df_binds, df_ss, datetime(2015, 1, 1))
df_binds_from_semi=bind_semi(df_semi,df_fbref,['mid','tid1','tid2','team1','team2'])
df_binds=pd.concat([df_binds,df_binds_from_semi], axis=0).drop_duplicates(subset=['ss_id','target_mid'])
df_binds.to_csv('data/binds_ss_fbref.csv', index=False)

In [ ]:
df_binds_from_semi=bind_semi(df_semi,df_fbref,['mid','tid1','tid2','team1','team2'])

In [ ]:
df_binds_from_semi

In [ ]:
pd.concat([df_binds,df_binds_from_semi], axis=0).drop_duplicates(subset=['ss_id','target_mid'])

## SS - OP

In [91]:
df_binds=bind_full(df_ss,df_op, datetime(2015, 1, 1), datetime(2020, 12, 1),'op',['link','t1','t2'])
print('*'*10,' Narrow approach ','*'*10)
df_ss_binded,df_semi,df_not=full_semi_none(df_binds, df_ss, datetime(2015, 1, 1))
df_binds1=bind_full(pd.concat([df_semi,df_not],axis=0).drop_duplicates(subset=['id']),df_op, datetime(2015, 1, 1), datetime(2020, 12, 1),'op',['link','t1','t2'], isWide=True)
df_binds=pd.concat([df_binds,df_binds1],axis=0)
print('*'*10,' Wide approach ','*'*10)
df_ss_binded,df_semi,df_not=full_semi_none(df_binds, df_ss, datetime(2015, 1, 1))

**********  Narrow approach  **********
 full: 28339, semi: 38317, non: 12080
**********  Wide approach  **********
 full: 28347, semi: 39046, non: 12080


In [88]:
df_semi

,awayTeam,homeScoreHT,homeTeam,ts,awayScoreFT,status,winnerCode,country,done,awayTeamShort,...,awayScoreET,countryCode,homeTeamShortLow,awayTeamShortLow,target_home_name,target_home_id,ss_home,target_away_name,target_away_id,ss_away
0,Queens Park Rangers,2.0,Burnley,2015-01-10 15:00:00+00:00,1.0,100.0,1.0,england,1,QPR,...,1.0,GB,burnley,qpr,Burnley,Burnley,Burnley,QPR,QPR,QPR
1,Queens Park Rangers,2.0,Burnley,2015-01-10 15:00:00+00:00,1.0,100.0,1.0,england,1,QPR,...,1.0,GB,burnley,qpr,Burnley U23,Burnley U23,Burnley,QPR,QPR,QPR
2,Newcastle United,1.0,Chelsea,2015-01-10 15:00:00+00:00,0.0,100.0,1.0,england,1,Newcastle,...,0.0,GB,chelsea,newcastle,Chelsea,Chelsea,Chelsea,Newcastle,Newcastle,Newcastle
3,Manchester City,0.0,Everton,2015-01-10 15:00:00+00:00,1.0,100.0,3.0,england,1,Man City,...,1.0,GB,everton,man city,Everton,Everton,Everton,Manchester City,Manchester City,Man City
4,Manchester City,0.0,Everton,2015-01-10 15:00:00+00:00,1.0,100.0,3.0,england,1,Man City,...,1.0,GB,everton,man city,Everton,Everton,Everton,Stoke City U23,Stoke City U23,Man City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50392,Trelleborgs FF,0.0,Jönköpings Södra,2020-11-28 14:00:00+00:00,1.0,100.0,1.0,sweden,1,Trelleborg,...,1.0,SE,jsodra,trelleborg,NaN,,NaN,Trelleborgs,Trelleborgs,Trelleborg
50393,Västerås SK,0.0,Umeå,2020-11-28 14:00:00+00:00,0.0,100.0,3.0,sweden,1,Västerås,...,0.0,SE,umea,vasteras,Umea FC,Umea FC,Umeå,Vasteras SK,Vasteras SK,Västerås
50394,Ljungskile SK,0.0,Östers IF,2020-11-28 14:00:00+00:00,1.0,100.0,2.0,sweden,1,Ljungskile,...,1.0,SE,oster,ljungskile,Osters,Osters,Öster,Ljungskile,Ljungskile,Ljungskile
50395,Vorskla Poltava,2.0,Dynamo Kyiv,2020-11-28 15:00:00+00:00,0.0,100.0,1.0,ukraine,1,Vorskla,...,0.0,UA,dynamo kyiv,vorskla,Dyn. Kyiv,Dyn. Kyiv,Dynamo Kyiv,Vorskla Poltava,Vorskla Poltava,Vorskla


In [92]:
df_binds_from_semi=bind_semi(df_semi,df_op,['link','t1','t2','t1','t2'])
df_binds=pd.concat([df_binds,df_binds_from_semi], axis=0).drop_duplicates(subset=['ss_id','target_mid'])
df_binds.to_csv('data/binds_ss_op1.csv', index=False)

df_ss_binded,df_semi,df_not=full_semi_none(df_binds, df_ss, datetime(2015, 1, 1))
df_binds_from_semi=bind_semi(df_semi,df_op,['link','t1','t2','t1','t2'],isWide=True)
df_binds=pd.concat([df_binds,df_binds_from_semi], axis=0).drop_duplicates(subset=['ss_id','target_mid'])
df_binds.to_csv('data/binds_ss_op2.csv', index=False)

df_ss_binded,df_semi,df_not=full_semi_none(df_binds, df_ss, datetime(2015, 1, 1))

Done! 5938 matches binded
 full: 34082, semi: 51263, non: 7560
Done! 960 matches binded
 full: 34738, semi: 80449, non: 6440


In [65]:
df_op

,ds,country,liga,season,t1,t2,sc1,sc2,odds1,oddsdraw,odds2,bn,link,done,home_name_low,away_name_low,ts
0,19/02/16 15:00,england,national-league-2018-2019,2018/2019,Boreham Wood,Hartlepool,0,4,2.29,3.26,3.09,12,/soccer/england/national-league-2018-2019/bore...,1,boreham wood,hartlepool,2019-02-16 15:00:00
1,14/03/25 19:00,germany,bundesliga-2013-2014,2013/2014,Dortmund,Schalke,0,0,1.46,4.82,6.42,7,/soccer/germany/bundesliga-2013-2014/dortmund-...,1,dortmund,schalke,2014-03-25 19:00:00
2,16/07/14 00:45,ecuador,serie-a-2016,2016,Barcelona SC,Guayaquil City,2,0,1.31,5.15,8.16,9,/soccer/ecuador/serie-a-2016/barcelona-sc-guay...,1,barcelona sc,guayaquil city,2016-07-14 00:45:00
3,17/04/08 14:00,england,league-two-2016-2017,2016/2017,Cambridge Utd,Leyton Orient,3,0,1.42,4.53,7.6,11,/soccer/england/league-two-2016-2017/cambridge...,1,cambridge utd,leyton orient,2017-04-08 14:00:00
4,16/09/20 15:00,iran,persian-gulf-pro-league-2016-2017,2016/2017,Esteghlal F.C.,Zob Ahan,2,1,1.9,2.94,4.59,9,/soccer/iran/persian-gulf-pro-league-2016-2017...,1,esteghlal fc,zob ahan,2016-09-20 15:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145372,18/12/01 14:30,germany,bundesliga-2018-2019,2018/2019,Stuttgart,Augsburg,1,0,2.66,3.39,2.71,12,/soccer/germany/bundesliga-2018-2019/vfb-stutt...,0,stuttgart,augsburg,2018-12-01 14:30:00
145373,19/12/08 16:30,croatia,1-hnl-2019-2020,2019/2020,Rijeka,Istra 1961,2,0,1.22,5.89,12.67,14,/soccer/croatia/1-hnl-2019-2020/rijeka-istra-1...,0,rijeka,istra 1961,2019-12-08 16:30:00
145374,16/08/14 20:30,peru,primera-division-2016,2016,Alianza Lima,Comerciantes Unidos,0,0,1.51,3.92,6.05,9,/soccer/peru/primera-division-2016/a-lima-come...,0,alianza lima,comerciantes unidos,2016-08-14 20:30:00
145375,10/11/20 15:00,england,league-one-2010-2011,2010/2011,Brighton,Bristol Rovers,2,2,1.45,4.34,6.56,6,/soccer/england/league-one-2010-2011/brighton-...,0,brighton,bristol rovers,2010-11-20 15:00:00


# Teams
Extract teams with Unicode 

In [ ]:
df_teams=pd.read_csv('data/teams_names.csv', index_col=None)
df_a=df_ss[['homeTeam','homeTeamShort','country','tournament']]
df_a.columns=['Team','TeamShort','country','tournament']
df_b=df_ss[['awayTeam','awayTeamShort','country','tournament']]
df_b.columns=['Team','TeamShort','country','tournament']
df_teams_new=pd.concat([df_a,df_b], axis=0).drop_duplicates()
df_teams_new['TeamEn']=df_teams_new['Team'].replace(dicUnicode2En, regex=True)
df_teams_new['TeamShortEn']=df_teams_new['TeamShort'].replace(dicUnicode2En, regex=True)
df_teams_new=df_teams_new.loc[df_teams_new['Team'].str.contains(r'[^\x00-\x7F]+')]
#df_teams_new['TeamEn']=''
#df_teams_new['TeamShortEn']=''

df_teams=pd.concat([df_teams,df_teams_new], axis=0).drop_duplicates(subset=['Team','country','tournament'], keep='first')

#df_teams_new.to_csv('data/teams_names1.csv', index=False)
df_teams.to_csv('data/teams_names.csv', index=False)

In [ ]:
df_teams

## Make binded SS list


In [ ]:
ds = pytz.timezone(local_tz).localize(datetime(2015, 5, 1))
ss=df_ss.loc[df_ss['ts']>=ds]

df_binds=pd.read_csv('data/binds.csv', index_col=None)
df_binds = df_binds.loc[~((df_binds['bf_home_name'].str.contains('\(Y\)')) | (df_binds['bf_home_name'].str.contains(' U20')) | (df_binds['bf_away_name'].str.contains('\(Y\)')) | (df_binds['bf_away_name'].str.contains(' U20'))) ]

df_ss_binded=ss.merge(df_binds, left_on=['id','homeTeamShort', 'awayTeamShort'] , right_on=['ss_id','ss_home','ss_away'], how='inner')
df_ss_binded=df_ss_binded.drop_duplicates()
df_semi = ss.loc[~(ss['id'].isin(df_ss_binded['id']))]

df_semi=df_semi.merge(df_binds[['bf_home_name', 'bf_home_id', 'ss_home']].drop_duplicates(), left_on=['homeTeamShort'] , right_on=['ss_home'], how='left')
df_semi=df_semi.merge(df_binds[['bf_away_name', 'bf_away_id', 'ss_away']].drop_duplicates(), left_on=['awayTeamShort'] , right_on=['ss_away'], how='left')
df_not=df_semi.loc[(df_semi['bf_home_id'].isna()) & (df_semi['bf_away_id'].isna())]
df_semi = df_semi.loc[~(df_semi['id'].isin(df_not['id']))]
df_semi=df_semi.drop_duplicates()
print(' full: {}, semi: {}, non: {}'.format(len(df_ss_binded.index),len(df_semi.index),len(df_not.index)) )

In [ ]:
binds=[]
binded_total=0
for row in df_semi.itertuples(index=False):
    d=row.ts.replace(tzinfo=None)
    if row.bf_home_id>0:
        #print(0,row.bf_home_id)
        df_bf_slice=df_bf[(df_bf['inplayTime']>=d) & (df_bf['inplayTime']<d+timedelta(days=1)) & (df_bf['home_id']==row.bf_home_id)]
        if len(df_bf_slice.index)==1:
            ssid, sshid, sshname, ssaname = (row.id,row.bf_home_id,row.homeTeamShort,row.awayTeamShort)
            bfid, bfhid, bfaid, bfhname, bfaname = (df_bf_slice.iloc[0][['eventId','home_id','away_id','home_name','away_name']]).values
            print('Found on home id={}, teams: {}/{} vs {}/{}, match ids {}-{}'.format(sshid, sshname,bfhname,ssaname,bfaname,ssid,bfid ))
            binds.append({
                'bf_home_name':bfhname,
                'bf_away_name':bfaname,
                'bf_home_id':bfhid,
                'bf_away_id':bfaid,
                'bf_eventId':bfid,
                'ss_home':sshname,
                'ss_away':ssaname,
                'ss_id':ssid
            })
            binded_total+=1
    else:
        #print(1,row.bf_away_id)
        df_bf_slice=df_bf[(df_bf['inplayTime']>=d) & (df_bf['inplayTime']<d+timedelta(days=1)) & (df_bf['away_id']==row.bf_away_id)]
        if len(df_bf_slice.index)==1:
            ssid, ssaid, sshname, ssaname = (row.id,row.bf_away_id,row.homeTeamShort,row.awayTeamShort)
            bfid, bfhid, bfaid, bfhname, bfaname = (df_bf_slice.iloc[0][['eventId','home_id','away_id','home_name','away_name']]).values
            print('Found on away id={}, teams: {}/{} vs {}/{}, match ids {}-{}'.format(ssaid, sshname,bfhname,ssaname,bfaname,ssid,bfid ))
            binds.append({
                'bf_home_name':bfhname,
                'bf_away_name':bfaname,
                'bf_home_id':bfhid,
                'bf_away_id':bfaid,
                'bf_eventId':bfid,
                'ss_home':sshname,
                'ss_away':ssaname,
                'ss_id':ssid
            })
            binded_total+=1
print (f'Done! {binded_total} matches binded')
pd.concat([df_binds,pd.DataFrame(binds)], axis=0).drop_duplicates(subset=['ss_id','bf_eventId']).to_csv('data/binds.csv', index=False)
    


In [ ]:
df_binds.drop_duplicates(subset=['ss_id','bf_eventId']).to_csv('data/binds.csv', index=False)

In [ ]:
df_ss_binded.drop_duplicates(subset=['ss_id','bf_eventId']).to_csv('data/matches_binded.csv', index=False)

In [ ]:
df_ss

In [ ]:
df=pd.concat([df1,df2], axis=0)

In [ ]:
df=pd.read_csv('data/matches_2015.csv', index_col=None)

In [ ]:
df.to_csv('data/matches_2015.csv', index=False)

In [ ]:
dfs=[]
for y in range(2015,2021):
    for f in glob.glob(path.join('data/bf/days/', f'{y}*_matches.csv')):
        print(f)
        dfs.append(pd.read_csv(f))
        
df=pd.concat(dfs, axis=0)
df=df.loc[~df['home_id'].isna()]
df=df[['eventId','countryCode','openDate','eventName','home_id','home_name','away_id','away_name','draw_id','draw_name','inplayTime','halfTime']]
df=df[df['eventName'].str.contains(' v ')]
df[['ht','at']]=df.apply(lambda x: x.eventName.split(' v '), axis=1, result_type="expand")
df=df.reset_index(drop=True)
df1=df.loc[(df['draw_name']!='The Draw')]
df2=df.loc[(df['draw_name']=='The Draw')]
df1.loc[(df1['away_name']=='The Draw') & (df1['ht'].str.lower()==df1['home_name'].str.lower()),'away_id']=df1['draw_id']
df1.loc[(df1['away_name']=='The Draw') & (df1['ht'].str.lower()==df1['home_name'].str.lower()),'away_name']=df1['at']
df1.loc[(df1['away_name']=='The Draw') & (df1['at'].str.lower()==df1['home_name'].str.lower()),'away_id']=df1['home_id']
df1.loc[(df1['away_name']=='The Draw') & (df1['at'].str.lower()==df1['home_name'].str.lower()),'home_id']=df1['draw_id']
df1.loc[(df1['away_name']=='The Draw') & (df1['at'].str.lower()==df1['home_name'].str.lower()),'home_name']=df1['ht']
df1.loc[(df1['away_name']=='The Draw') & (df1['ht'].str.lower()==df1['home_name'].str.lower()),'away_name']=df1['at']
df1.loc[(df1['home_name']=='The Draw') & (df1['at'].str.lower()==df1['away_name'].str.lower()),'home_id']=df1['draw_id']
df1.loc[(df1['home_name']=='The Draw') & (df1['at'].str.lower()==df1['away_name'].str.lower()),'home_name']=df1['ht']
df1.loc[(df1['home_name']=='The Draw') & (df1['ht'].str.lower()==df1['away_name'].str.lower()),'home_id']=df1['away_id']
df1.loc[(df1['home_name']=='The Draw') & (df1['ht'].str.lower()==df1['away_name'].str.lower()),'away_id']=df1['draw_id']
df1.loc[(df1['home_name']=='The Draw') & (df1['ht'].str.lower()==df1['away_name'].str.lower()),'away_name']=df1['at']
df1.loc[(df1['home_name']=='The Draw') & (df1['at'].str.lower()==df1['away_name'].str.lower()),'home_name']=df1['ht']
df=pd.concat([df1,df2], axis=0)
df.dropna(subset=['inplayTime'], inplace=True)
df=df[~((df['home_name']=='test1') | (df['away_name']=='test1') | (df['home_name']=='test2') | (df['away_name']=='test2'))]
df=df[['eventId','countryCode','openDate','eventName','home_id','home_name','away_id','away_name','inplayTime','halfTime']]
df.to_csv('data/bf_matches.csv', index=False)

In [ ]:
df.to_csv('data/bf_matches.csv', index=False)

In [ ]:
cols=['eventId','clk','ltp','id','ip']
cols_noid=['eventId','clk','ltp','ip']
df_matches=pd.read_csv('data/bf/bf_matches.csv', index_col=None)
for y in range(2015,2021):
    dfs=[]
    for f in glob.glob(path.join('data/bf/days/', f'{y}*_odds.csv')):
        print(f)
        dfs.append(pd.read_csv(f))
        
    df=pd.concat(dfs, axis=0)
    df=df[df['eventId'].isin(df_matches['eventId'])]
    df['ip']=0
    df.loc[df['inplay'],'ip']=1
    df=df[cols]
    df_un=df.loc[df['id']==ID_UNDER][cols_noid]
    df_ov=df.loc[df['id']==ID_OVER][cols_noid]
    df_draw=df.loc[df['id']==ID_DRAW][cols_noid]
    df_home=pd.merge(df,df_matches, left_on=['eventId','id'], right_on=['eventId','home_id'])[cols_noid]
    df_away=pd.merge(df,df_matches, left_on=['eventId','id'], right_on=['eventId','away_id'])[cols_noid]
    #df_12=df.loc[(df['id']!=ID_UNDER) & (df['id']!=ID_OVER) & (df['id']!=ID_DRAW)]
    df_un.to_csv(f'data/bf_un_{y}.csv', index=False)
    df_ov.to_csv(f'data/bf_ov_{y}.csv', index=False)
    df_draw.to_csv(f'data/bf_draw_{y}.csv', index=False)
    df_home.to_csv(f'data/bf_home_{y}.csv', index=False)
    df_away.to_csv(f'data/bf_away_{y}.csv', index=False)
    #break

In [ ]:
df_matches=pd.read_csv('data/bf_matches.csv', index_col=None)

